In [46]:
from torch.utils.data import Dataset, DataLoader
import torch
from tqdm import tqdm
import torch.nn.functional as F
from transformers import BertForTokenClassification, BertTokenizer

In [47]:
tag_set = []
# 打开文件并逐行读取
with open('tag_set.txt', 'r', encoding='utf-8') as file:
    for line in file:
        # 分割每一行中的标签 再通过空格分隔
        tags = line.strip().split()
        # 更新集合，自动去除重复的标签
        tag_set += tags
tag_set += ['pad']

num_labels = len(tag_set)

# 创建标签到索引的映射
label2idx = {label: idx for idx, label in enumerate(tag_set)}

In [48]:
test_texts = []

# 读取文本文件
with open('data/test.txt', 'r', encoding='utf-8') as file_texts:
    test_texts = [line.strip().replace(' ', '') for line in file_texts if line.strip()]

print("示例文本：", test_texts[0])

示例文本： 记者1月15日从上海铁路局淮南西站获悉,淮南铁路预计春运期间发送旅客33.3万人。预计客流最高峰日为2月6日,将发送旅客1.8万人,淮南东开往广州南及北京方向的高铁、淮南站开往沪杭甬方向的列车将是热门车次。


In [49]:
# 计算每个字符串的长度
lengths = [len(text) for text in test_texts]

# 计算平均长度
average_length = sum(lengths) / len(lengths)

# 找到最短和最长的字符串长度
min_length = min(lengths)
max_length = max(lengths)

# 生成长度的频率分布
from collections import Counter
length_distribution = Counter(lengths)

print(f"平均长度: {average_length}")
print(f"最短长度: {min_length}, 最长长度: {max_length}")
print("长度频率分布:")
beyond = 0
for length, count in sorted(length_distribution.items()):
    print(f"长度 {length}: 出现次数 {count}")
    if length > 128:
        beyond += count
print(beyond)

平均长度: 83.4152832774901
最短长度: 1, 最长长度: 1457
长度频率分布:
长度 1: 出现次数 41
长度 2: 出现次数 206
长度 3: 出现次数 148
长度 4: 出现次数 275
长度 5: 出现次数 326
长度 6: 出现次数 294
长度 7: 出现次数 332
长度 8: 出现次数 349
长度 9: 出现次数 335
长度 10: 出现次数 364
长度 11: 出现次数 375
长度 12: 出现次数 368
长度 13: 出现次数 349
长度 14: 出现次数 323
长度 15: 出现次数 294
长度 16: 出现次数 252
长度 17: 出现次数 316
长度 18: 出现次数 243
长度 19: 出现次数 419
长度 20: 出现次数 362
长度 21: 出现次数 407
长度 22: 出现次数 435
长度 23: 出现次数 274
长度 24: 出现次数 290
长度 25: 出现次数 255
长度 26: 出现次数 229
长度 27: 出现次数 224
长度 28: 出现次数 162
长度 29: 出现次数 108
长度 30: 出现次数 134
长度 31: 出现次数 91
长度 32: 出现次数 92
长度 33: 出现次数 101
长度 34: 出现次数 105
长度 35: 出现次数 83
长度 36: 出现次数 88
长度 37: 出现次数 83
长度 38: 出现次数 69
长度 39: 出现次数 79
长度 40: 出现次数 104
长度 41: 出现次数 91
长度 42: 出现次数 103
长度 43: 出现次数 74
长度 44: 出现次数 99
长度 45: 出现次数 128
长度 46: 出现次数 95
长度 47: 出现次数 143
长度 48: 出现次数 109
长度 49: 出现次数 103
长度 50: 出现次数 95
长度 51: 出现次数 124
长度 52: 出现次数 129
长度 53: 出现次数 129
长度 54: 出现次数 112
长度 55: 出现次数 130
长度 56: 出现次数 143
长度 57: 出现次数 128
长度 58: 出现次数 133
长度 59: 出现次数 142
长度 60: 出现次数 131
长度 61: 出现次数

In [50]:
# 切割函数
def split_text(texts, batchsize=128):
    split_texts = []    # 存放切割后的文本段落
    sign = []           # 存放每个文本段落的标记，以便后面合并分割的段落
    count = 0           # 用于记录当前文本段落的索引
    for text in texts:
        if len(text) > batchsize:
            num = int(len(text) / batchsize)
            # 如果文本长度超过了指定的批次大小，则进行切割
            for i in range(num):
                # 将文本按照批次大小进行切割，添加到切割后的文本列表中
                split_texts.append(text[batchsize * i: batchsize * (i + 1)])
                # 记录当前文本段落的标记，方便后面合并分割的段落
                sign.append(count)
            # 将剩余的部分作为最后一个段落
            split_texts.append(text[batchsize * num:])
            sign.append(count)
        else:
            # 如果文本长度未超过批次大小，则不进行切割，直接添加到切割后的文本列表中
            split_texts.append(text)
            sign.append(count)
        
        count += 1  # 更新文本段落的索引
    
    return split_texts, sign

# 使用函数切割文本
split_test_texts, sign = split_text(test_texts)

In [51]:
# 数据准备
class Dataset(Dataset):
    def __init__(self, texts, tokenizer, max_len):
        self.texts = texts          # 保存传入的文本数据列表
        self.tokenizer = tokenizer  # 保存分词器对象
        self.max_len = max_len      # 设置模型输入的最大序列长度

    # 返回数据集中文本的数量
    def __len__(self):
        return len(self.texts)

    # 通过索引获取数据集中的一个样本
    def __getitem__(self, idx):
        text = self.texts[idx]  # 根据索引获取单个文本字符串
        
        encoding = self.tokenizer(
            text,
            padding='max_length',   # 不足max_len的部分使用pad填充
            truncation=True,        # 超过max_len的部分进行截断
            max_length=self.max_len,  # 设置最大长度
            return_tensors='pt'         # 返回pytorch张量
        )
        

        # input_ids: 编码后的token ID序列 表示文本的数字形式
        # attention_mas': 注意力掩码 与input_ids长度相同
        return {
            'input_ids': encoding['input_ids'][0],
            'attention_mask': encoding['attention_mask'][0]
        }

In [52]:
# 指定之前保存的目录路径
saved_tokenizer_directory = './trains/train1/bert_tokenizer/'
saved_model_directory = './trains/train1/bert_model/'

# 重新加载分词器
tokenizer = BertTokenizer.from_pretrained(saved_tokenizer_directory)

# 重新加载模型
model = BertForTokenClassification.from_pretrained(saved_model_directory)

# 现在，tokenizer和model已经加载完毕，可以用于文本处理和推理等任务

In [53]:
test_dataset = Dataset(split_test_texts, tokenizer, max_len=128)

In [54]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def tag_test(model, test_dataset, device):
    '''对 test.txt 进行序列标注，并显示测试进度'''
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)  # 验证时不需要shuffle

    model.eval()  # 设置模型为评估模式
    model.to(device)

    # 存放所有的句子的结果
    result = []

    with torch.no_grad():  # 禁用梯度计算
        # 使用tqdm包装数据加载器以显示进度
        for batch in tqdm(test_loader, desc="Testing", unit="batch", leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            # 模型前向传播
            outputs = model(input_ids, attention_mask=attention_mask)
            
            # 获取概率分布并预测类别
            prob = F.softmax(outputs.logits, dim=-1)
            preds = torch.argmax(prob, dim=-1)

            # 收集预测结果
            result.extend(preds.tolist())  # 直接扩展列表，无需内部循环

    return result

In [55]:
results = tag_test(model, test_dataset, device)

In [57]:
len(results), len(split_test_texts), len(sign)

(32843, 32843, 32843)

In [66]:
label2idx['pad']

9

In [73]:
# 初始化一个空列表，用于存储过滤后的结果
filtered_result = []

# 遍历原始结果列表中的每个样本
for result in results:
    # 遍历当前样本中的每个元素 去除pad对应的索引
    f_result = [num for num in result if num != label2idx['pad']]
    filtered_result.append(f_result)

In [77]:
len(filtered_result)

32843

In [87]:
def merge_elements(filtered_result, sign):
    # 初始化一个字典来存储合并后的结果
    merged_dict = {}
    # 遍历sign和filtered_result，根据sign的值合并filtered_result的元素
    for index, value in enumerate(sign):
        if value not in list(merged_dict.keys()):
            # 如果sign的值首次出现，直接赋值
            merged_dict[value] = filtered_result[index]
        else:

            # 如果sign的值重复出现，合并filtered_result的元素（这里假设是列表相加）
            merged_dict[value].extend(filtered_result[index])
    
    # 确保sign中的值排序与原列表一致，然后根据这个顺序重组merged_result
    sorted_sign = sorted(merged_dict.keys())
    merged_result = [merged_dict[val] for val in sorted_sign]
    
    return merged_result

# 调用函数
merged_list = merge_elements(filtered_result, sign)
print(len(merged_list))

26264


In [93]:
def listTotags(merged_list):
    '''将列表中的数字转化为tags标签'''
    merged_tags = []
    for sentence in merged_list:
        tags = [tag_set[num] for num in sentence]
        # 将一个长文本的标签合并到一起 中间用空格隔开
        tags = ' '.join(tags)
        merged_tags.append(tags)

    return merged_tags

In [94]:
merged_tags = listTotags(merged_list)

In [95]:
merged_tags[0]

'O O B_T I_T I_T I_T I_T O B_LOC I_LOC O O O B_LOC I_LOC O O O O O B_LOC I_LOC O O O O O O O O O O O O O O O O O O O O O O O O O O O O B_T I_T I_T I_T O O O O O O O O O O O O B_LOC I_LOC O O O B_LOC I_LOC O O B_LOC I_LOC O O O O O O B_LOC I_LOC O O O O O O O O O O O O O O O O O O'

In [96]:
test_texts[0]

'记者1月15日从上海铁路局淮南西站获悉,淮南铁路预计春运期间发送旅客33.3万人。预计客流最高峰日为2月6日,将发送旅客1.8万人,淮南东开往广州南及北京方向的高铁、淮南站开往沪杭甬方向的列车将是热门车次。'

In [97]:
with open('2021213346.txt', 'w') as file:
    # 遍历列表，将每个元素写入文件，每个元素后跟一个换行符('\n')
    for tag in merged_tags:
        file.write(tag + '\n')